In [2]:
import pandas as pd
import datetime
import seaborn as sns


In [3]:
def utc_timestamp_to_date(utc_timestamp):
    return datetime.datetime.utcfromtimestamp(utc_timestamp).date()

In [4]:
file_path_comment = 'modules/data/currencies/bitcoin/reddit-r-bitcoin-data-for-jun-2022-comments.csv'
file_path_post = 'modules/data/currencies/bitcoin/reddit-r-bitcoin-data-for-jun-2022-posts.csv'


In [5]:
dfC = pd.read_csv(file_path_comment)
dfP = pd.read_csv(file_path_post)
dfC = dfC.sort_values(by='created_utc', ascending=True)
dfP = dfP.sort_values(by='created_utc', ascending=True)


dfC.head()  
dfP.head()


,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
7540,post,v23s21,2s3qj,bitcoin,False,1654041687,https://old.reddit.com/r/Bitcoin/comments/v23s...,self.bitcoin,NaN,I have nothing against Gemini I will definitel...,finally took all my coins off exchange into a ...,127
7539,post,v23s5e,2s3qj,bitcoin,False,1654041696,https://old.reddit.com/r/Bitcoin/comments/v23s...,self.bitcoin,NaN,[removed],How can I invest btc,0
7538,post,v24ekx,2s3qj,bitcoin,False,1654043678,https://old.reddit.com/r/Bitcoin/comments/v24e...,thehificrypto.substack.com,https://thehificrypto.substack.com/p/bitcoin-a...,NaN,Don’t fall for Ponzis and scams. Protect your ...,9
7537,post,v24v41,2s3qj,bitcoin,False,1654045152,https://old.reddit.com/r/Bitcoin/comments/v24v...,self.bitcoin,NaN,[removed],https://www.digistore24.com/redir/299134/Sadac...,1
7536,post,v24vhp,2s3qj,bitcoin,False,1654045187,https://old.reddit.com/r/Bitcoin/comments/v24v...,self.bitcoin,NaN,[removed],school coin,0


In [6]:
dfC['created_date'] = dfC['created_utc'].apply(utc_timestamp_to_date)
dfP['created_date'] = dfP['created_utc'].apply(utc_timestamp_to_date)
dfC.drop(['created_utc','subreddit.id','subreddit.name', 'subreddit.nsfw','permalink','type','sentiment'], axis=1, inplace=True)
dfP.drop(['created_utc','subreddit.id','subreddit.name', 'subreddit.nsfw','permalink','type','domain','url','selftext'], axis=1, inplace=True)

dfP.head()

,id,title,score,created_date
7540,v23s21,finally took all my coins off exchange into a ...,127,2022-06-01
7539,v23s5e,How can I invest btc,0,2022-06-01
7538,v24ekx,Don’t fall for Ponzis and scams. Protect your ...,9,2022-06-01
7537,v24v41,https://www.digistore24.com/redir/299134/Sadac...,1,2022-06-01
7536,v24vhp,school coin,0,2022-06-01


In [42]:
dfC.head()
print(dfC.shape)

(169820, 4)


In [8]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [29]:
pipe = pipeline("text-classification", model="mwkby/distilbert-base-uncased-sentiment-reddit-crypto")
tokenizer = AutoTokenizer.from_pretrained("mwkby/distilbert-base-uncased-sentiment-reddit-crypto")
model = AutoModelForSequenceClassification.from_pretrained("mwkby/distilbert-base-uncased-sentiment-reddit-crypto")

In [33]:
max_seq_length = 512  # Adjust as needed

# Filter rows based on the sequence length of the 'comments' column
dfC = dfC[dfC['body'].apply(lambda x: len(tokenizer.encode(x)) <= max_seq_length)]

# Now, 'dfC' contains only the rows where 'comments' fits within 'max_seq_length'

In [34]:
def polarity_scores(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'finbert_neg' : scores[0],
        'finbert_pos' : scores[1]
    }
    return scores_dict

In [44]:
res = {}
for i, row in dfC.iterrows():
    text = row['body']
    myid = row['id']
    res[myid] = polarity_scores(text)

[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:109] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

It takes around 100 mins to analyse 1 month in 1 currency subreddit.
It should take around 1 day to analyse 5 year.

In [41]:
distilbert = pd.DataFrame(res).T
distilbert = distilbert.reset_index().rename(columns={'index': 'Comment ID'})
print(distilbert.shape)

(400, 3)
